In [3]:
import pandas as pd
from collections import defaultdict

    # Import csv file with the coordinates in (x,y,z) of all points
coor = pd.read_csv('coordenadas.csv')
coor.set_index('Punto', inplace=True)

    # Declaration of a "graph" to assign edges and weights
class Graph():
    def __init__(self):
        self.edges = defaultdict(list)
        self.weights = {}
    
    def add_edge(self, from_node, to_node, weight):
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight
        
    
def dijsktra(initial, end, max):
    # Import matrix of points
    df = pd.read_csv('matriz.csv')
    df.set_index('Punto', inplace=True)

    col = df.columns[:]
    valores = []

    # Creation of a list "valores" with the distance between points
    i = 2
    for column in df.columns[:]:
        while i <= 177:
            valores.append([int(column),i,df[column][i]])
            i += 1
        i = int(column)+1

    # "grafo" filters the points in "valores" with a distance higher than "max", assigned by the user
    grafo = [valores[i] for i in range(len(valores)) if valores[i][2] < max]

    graph = Graph()
    edges = grafo

    for edge in edges:
        graph.add_edge(*edge)

    # Dijkstra algorithm
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()
    weight = 0
    
    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)
        
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
        
    # Work back through destinations in shortest path
    path = []
    res = []
    cost = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        cost.append(shortest_paths[current_node][1])
        current_node = next_node
    # Reverse path
    path = path[::-1]
    res.append(path)
    res.append(round(cost[0],2))
    return res

    # Final function
def dijkstra_brazo(*args):
    val = []
    res = []
    dist = 0
    for i in args:
        val.append(i)
    for i in range(0, len(val)-2):
        res = dijsktra(val[i],val[i+1],val[-1])
        dist = dist + res[1]
        print(str(res)+",")
    print("Total distance(mm): "+str(round(dist,2)))
    #%%time
    
    # Function to find the coordinates in (x,y,z) of points
def coordenadas(*args):
    for i in args:
        x = coor['X'][i]
        y = coor['Y'][i]
        z = coor['Z'][i]
        print('Punto '+str(i)+': x: '+str(x)+', y: '+str(y)+', z: '+str(z))